# Analiza danych przestrzennych - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania i oceny notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Jeżeli zestaw zadań wymaga skorzystania z fragmentów kodu opracowanego w ramach wcześniejszych zestawów zadań należy je umieścić we wskazanej komórce notatnika.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu, odpowiedzi uzupełniane manualnie nie podlegają ocenie.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają na celu wyłapać podstawowe błędy. Przejście przez kod wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie i zostanie ocenione na maksymalną liczbę punktów.
- Zadania należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zadania, które powodują wyświetlenie komunikatu o błędzie przerywającym wykonywanie kodu nie podlegają ocenie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem). Rozmiar poziomy figur nie powinien przekraczać 20 cali.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu: Zuzanna Brauer, Sandra Głowacka, Zuzanna Chmielarska <br>
Członkowie: IAD13

---

## Zestaw zadań 4: Wybrane teoretyczne rozkłady prawdopodobieństwa oraz elementy teorii estymacji (Notatnik 2/3)

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
from math import sqrt

ModuleNotFoundError: No module named 'sklearn'

### Dane do zadań

W celu wygenerowania danych wykorzystywanych w zawartych w notatniku zadaniach i komórkach testowych wykonaj obie poniższe komórki.

In [3]:
# Dane do testów 1
test_data_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=20, random_state=7), columns=["X"])
test_data_1.head()

,X
0,5.338105
1,4.906813
2,5.006564
3,5.081503
4,4.842215


In [4]:
# Dane do testów 2
x = sp.stats.uniform.rvs(loc=-2, scale=10, size=250, random_state=34)
y = 2*x - 5 + sp.stats.norm.rvs(loc=0, scale=2, size=250, random_state=13)
test_data_2 = pd.DataFrame(data=np.array([x, y]).T, columns=["X", "Y"])
test_data_2.head()

,X,Y
0,-1.614383,-9.653548
1,5.801005,8.109542
2,-1.072962,-7.234931
3,4.328927,4.561478
4,-1.861092,-6.031980


### Zadanie 3: Estymacja przedziałowa wartości oczekiwanej [7 pkt]

Przygotuj funkcję `mean_estimation()`, która będzie dokonywała estymacji przedziałowej wartości oczekiwanej dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`𝜇 należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [7]:
def mean_estimation(data, alpha, population_std="unknown"):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i pojedynczą kolumnę "X" z wartościami próby losowej,
    alpha: float
        Wartość określająca poziom istotności.
    population_std: str or float
        Jeżeli odchylenie standardowe populacji nie jest znane to parametr przyjmuje wartość "unknown",
        w przeciwnym wypadku jest to wartość określająca odchylenie standardowe populacji.
    
    Returns
    -------
    mean_low: float
        Dolna granica wyliczonego przedziału ufności.
    mean_high: float
        Górna granica wyliczonego przedziału ufności.
    """
   
    n = len(data)
    data = data.to_numpy() # zamiana formatu ponieważ problem z printem przy pd.Series
    mean = np.mean(data)

    if population_std == "unknown":
        t_score = sp.stats.t.ppf(1 - alpha/2, df = n-1)
        margin = t_score*np.std(data, ddof = 1)/np.sqrt(n)
    else:
        t_score = sp.stats.norm.ppf(1 - alpha/2)
        margin = t_score*population_std /np.sqrt(n)
        
    mean_low = mean - margin
    mean_high = mean + margin
    print(f"𝜇 należy do przedziału [{mean_low:.4f}, {mean_high:.4f}] przy założeniu poziomu ufności 1 - {alpha} = {1 - alpha:.2f}")

    return (mean_low,mean_high)

In [8]:
### Komórka testowa

# Test na test_data_1 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.1, population_std=0.2), (4.905619719495418, 5.05273990041144))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.05, population_std=0.2), (4.891527555895771, 5.066832064011087))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.02, population_std=0.2), (4.875142370240081, 5.083217249666777))))
print()

# # Test na test_data_1 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.1), (4.913356369231727, 5.045003250675131))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.05), (4.899504007229391, 5.058855612677467))))
print()
assert(np.all(np.isclose(mean_estimation(test_data_1, 0.02), (4.882508507351398, 5.07585111255546))))

𝜇 należy do przedziału [4.9056, 5.0527] przy założeniu poziomu ufności 1 - 0.1 = 0.90

𝜇 należy do przedziału [4.8915, 5.0668] przy założeniu poziomu ufności 1 - 0.05 = 0.95

𝜇 należy do przedziału [4.8751, 5.0832] przy założeniu poziomu ufności 1 - 0.02 = 0.98

𝜇 należy do przedziału [4.9134, 5.0450] przy założeniu poziomu ufności 1 - 0.1 = 0.90

𝜇 należy do przedziału [4.8995, 5.0589] przy założeniu poziomu ufności 1 - 0.05 = 0.95

𝜇 należy do przedziału [4.8825, 5.0759] przy założeniu poziomu ufności 1 - 0.02 = 0.98


### Zadanie 4: Estymacja przedziałowa wariancji [5 pkt]
Przygotuj funkcję `variance_estimation()`, która będzie dokonywała estymacji przedziałowej wariancji dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`𝜎^2  należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [9]:
def variance_estimation(data, alpha):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i pojedynczą kolumnę "X" z wartościami próby losowej,
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    var_low: float
        Dolna granica wyliczonego przedziału ufności.
    var_high: float
        Górna granica wyliczonego przedziału ufności.
    """
    
    x = data.to_numpy()
    n = len(x)
    mean = np.mean(data)
    var = np.sum((x - mean)*(x - mean))/n
    var_low = var*n/sp.stats.chi2.ppf(1 - alpha/2, n - 1)
    var_high = var*n/sp.stats.chi2.ppf(alpha/2, n - 1)
    print(f"𝜎^2 należy do przedziału [{var_low:.4f}, {var_high:.4f}] przy założeniu poziomu ufności 1 - {alpha} = {1 - alpha:.2f}")
    
    return (var_low,var_high)

In [10]:
### Komórka testowa
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.1), (0.018268125369359626, 0.05442967510189219))));
print()
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.05), (0.0167618487538223, 0.06182728513230527))));
print()
assert(np.all(np.isclose(variance_estimation(test_data_1, 0.02), (0.015215598500840796, 0.07214532146342346))));

𝜎^2 należy do przedziału [0.0183, 0.0544] przy założeniu poziomu ufności 1 - 0.1 = 0.90

𝜎^2 należy do przedziału [0.0168, 0.0618] przy założeniu poziomu ufności 1 - 0.05 = 0.95

𝜎^2 należy do przedziału [0.0152, 0.0721] przy założeniu poziomu ufności 1 - 0.02 = 0.98


### Zadanie 5: Estymacja wartości współczynnika korelacji liniowej [5 pkt]

Przygotuj funkcję `correlation_estimation()`, która będzie dokonywała estymacji przedziałowej wartości współczynnika korelacji liniowej Pearsona dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`r należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [60]:
def correlation_estimation(data, alpha):

    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i dwie kolumny "X" i "Y" z wynikami próby losowej.
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    r_corr_low: float
        Dolna granica wyliczonego przedziału ufności.
    r_corr_high: float
        Górna granica wyliczonego przedziału ufności.
    """   

    r = sp.stats.pearsonr(data['X'], data['Y']).statistic
    r_corr_low = r - sp.stats.norm.ppf(1 - alpha/2)*(1 - r*r)/sqrt(len(data))
    r_corr_high = r + sp.stats.norm.ppf(1 - alpha/2)*(1 - r*r)/sqrt(len(data))
    print(f"r należy do przedziału [{r_corr_low:.4f}, {r_corr_high:.4f}] przy założeniu poziomu ufności 1 - {alpha} = {1 - alpha:.2f}")

    return(r_corr_low, r_corr_high)

In [61]:
### Komórka testowa
assert np.all(np.isclose(correlation_estimation(test_data_2, 0.05), (0.9494696249316307, 0.9692185148958979)))
assert np.all(np.isclose(correlation_estimation(test_data_2, 0.02), (0.9476237555239514, 0.9710643843035772)))

r należy do przedziału [0.9495, 0.9692] przy założeniu poziomu ufności 1 - 0.05 = 0.95
r należy do przedziału [0.9476, 0.9711] przy założeniu poziomu ufności 1 - 0.02 = 0.98


### Zadanie 6: Estymacja współczynników równania regresji liniowej [8 pkt]

Przygotuj funkcję `interval_linear_regression_coefficients_estimation()`, która będzie dokonywała estymacji przedziałowej wartości współczynników równania regresji liniowej dla danych wejściowych w postaci szeregu szczegółowego zgodnie ze schematem z załączonego do notatnika zestawu wzorów.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`a należy do przedziału [X1, Y1] przy założeniu poziomu ufności 1-𝛼 = Z1` </br>
`b należy do przedziału [X2, Y2] przy założeniu poziomu ufności 1-𝛼 = Z2`

gdzie X1, X2, Y1, Y2, Z1 i Z2 są automatycznie uzupełniane przez funkcję, a X1, X2, Y1 i Y2 dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [50]:
def linear_regression_coefficients_estimation(data, alpha):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i dwie kolumny "X" i "Y" z wynikami próby losowej.
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    a_ci: tuple
        Zmienna typu tuple zawierajaca granice przedziału ufności parametru a (a_low, a_high).
    b_ci: tuple
        Zmienna typu tuple zawierajaca granice przedziału ufności parametru b (b_low, b_high).
    """   

    reg = sp.stats.linregress(data['X'], data['Y'])
    a_low = reg.slope - sp.stats.t.ppf(1 - alpha/2, df = len(data) - 2)*reg.stderr
    a_high = reg.slope + sp.stats.t.ppf(1 - alpha/2, df = len(data) - 2)*reg.stderr
    b_low = reg.intercept - sp.stats.t.ppf(1 - alpha/2, df = len(data) - 2)*reg.intercept_stderr
    b_high = reg.intercept + sp.stats.t.ppf(1 - alpha/2, df = len(data) - 2)*reg.intercept_stderr
    a_ci = (a_low, a_high)
    b_ci = (b_low, b_high)
    print(f"a należy do przedziału [{a_low:.4f}, {a_high:.4f}] przy założeniu poziomu ufności 1 - {alpha} = {1 - alpha:.2f}")
    print(f"b należy do przedziału [{b_low:.4f}, {b_high:.4f}] przy założeniu poziomu ufności 1 - {alpha} = {1 - alpha:.2f}")
    
    return(a_ci, b_ci)

In [51]:
### Komórka testowa
assert np.all(np.isclose(linear_regression_coefficients_estimation(test_data_2, 0.1), ((1.90330932143819, 2.024457061652292), (-5.331039505544365, -4.7958291264947706))))
print()
assert np.all(np.isclose(linear_regression_coefficients_estimation(test_data_2, 0.05), ((1.8916219551002786, 2.0361444279902035), (-5.382672328053067, -4.744196303986069))))

a należy do przedziału [1.9033, 2.0245] przy założeniu poziomu ufności 1 - 0.1 = 0.90
b należy do przedziału [-5.3310, -4.7958] przy założeniu poziomu ufności 1 - 0.1 = 0.90

a należy do przedziału [1.8916, 2.0361] przy założeniu poziomu ufności 1 - 0.05 = 0.95
b należy do przedziału [-5.3827, -4.7442] przy założeniu poziomu ufności 1 - 0.05 = 0.95
